In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.types import Integer
import math
import keyring
from pprint import pprint
import requests
import json
import folium

# set connection parameters
dbName = 'happy_road'
dbUser = 'orms0027'
dbPass = keyring.get_password('tpostgres', 'orms0027')
dbHost = 'usvcs-tpostgres.uservices.umn.edu'

In [2]:
# convert GeoJson coordinate list into a properly formatted 
# strings to pass to OSRM API
def geojson_to_OSRM_strings(geojson):
    
    #generate a coordinate pair string
    coord_separator = ','
    coord_pair_separator = ';'
    coord_list = []
    for coord_pair in geojson['coordinates']:
        coord_pair_string = coord_separator.join([str(coord) for coord in coord_pair])
        coord_list.append(coord_pair_string)
    coord_pairs_string = coord_pair_separator.join(coord_list)
    
    # generate a timestamp string
    timestamp_separator = ';'
    timestamp_string_list = []
    for timestamp in geojson['properties']['timestamps']:
        timestamp_string_list.append(str(int(timestamp / 1000)))
    timestamps_string = timestamp_separator.join(timestamp_string_list)
    
    # generate a string of accuracies to use as a search radius
    
    radius_separator = ';'
    radius_string_list = []
    for radius in geojson['properties']['accuracies']:
        radius_string_list.append(str(radius))
    radiuses_string = radius_separator.join(radius_string_list)

    
    return coord_pairs_string, timestamps_string, radiuses_string

In [3]:
# returns the arguments needed to pass a match request to the docker instance of OSRM
# see OSRM backend documentation at https://github.com/Project-OSRM/osrm-backend

def match(coord_string,
          mode='biking', 
          steps='true',
          tidy='true',
          geometries='geojson',
          gaps='ignore',
          overview='false', 
          annotations='nodes',
          timestamps=False,
          radiuses=False):

    url = 'http://localhost:5000/match/v1/{}/{}?steps={}&geometries={}&tidy={}&gaps={}'.format(mode, coord_string, steps, geometries, tidy, gaps)

    if overview:
        url += '&overview={}'.format(overview)
    if annotations:
        url += '&annotations={}'.format(annotations)
    if radiuses:
        url += '&radiuses={}'.format(radiuses)
    if timestamps:
        url +='&timestamps={}'.format(timestamps)
    
    return url

In [4]:
# given a URL string suitable for passing to OSRM, return the JSON output from OSRM
def match_json(url):
    # gets the returned json from OSRM
    r = requests.get(url)
    
    # convert to dictionary
    matchings_dict = json.loads(r.text)
    
    return matchings_dict

In [5]:
# takes a trip id, the URL from the match function, and a python dict in GeoJson feature collection
# format.  Passes the URL to OSRM which returns a JSON string, which includes the matchings.  
# Converts the JSON string to a feature in GeoJson format for each matching, and adds it to 
# the feature collection dictionary.  Returns the feature collection dictionary.

# the index option is the starting feature id value

# the timestamps value is a list of the times for the input GPS points

def matchings_geojson(tid, url, feature_collection, timestamps, index=0):
    
    matchings_dict = match_json(url)
    
    try:
        # convert the matchings from the dictionary into GeoJson formatted features
        for matching in matchings_dict['matchings']:
            confidence = matching['confidence']
            # initialize a list to store all the timestamps associated with the matching
            timestamps_list = []  
            feature = {
                       'type': 'Feature',
                       'id': index,
                       'geometry': {
                                    'type': 'LineString',
                                    'coordinates': []
                                   },
                       'properties': {
                                    'start_time': None,
                                    'end_time': None,
                                    'confidence': confidence,
                                    'tid': tid
                                    }
                      }

            # loop through the legs and steps of the matching to get a single set of coords for the whole matching
            for leg in matching['legs']:

                # each input GPS point defines a leg in the matched route, so popping the timestamp for each leg into
                # the timestamp list will make a list of all timestamps associated with a matching
                timestamps_list.append(timestamps.pop(0))  

                for step in leg['steps']:
                    feature['geometry']['coordinates'].extend(step['geometry']['coordinates'])

            feature['properties']['start_time'] = timestamps_list[0]

            # the last timestamp in the list is the start time of the last leg.  Add 5 seconds to approximate the actual end time
            feature['properties']['end_time'] = timestamps_list[-1] + 5000 

            # put the feature representing the matching into the feature collection
            feature_collection['features'].append(feature)

            # advance the feature id index 
            index += 1
            
    except:
        print('error during trip {}: {}'.format(tid, matchings_dict['message']))
    
    return feature_collection, index + 1  # return the feature collection and the index of the next feature to add to the collection

In [6]:
# takes a trip id, the URL from the match function, and a python dict in GeoJson feature collection
# format.  Passes the URL to OSRM which returns a JSON string, which includes the matchings.  
# Converts the JSON string to a point feature in GeoJson format for the first point in each leg, and adds it to 
# the feature collection dictionary.  Returns the feature collection dictionary.

# the index option is the starting feature id value

# the timestamps value is a list of the times for the input GPS points

def leg_points_geojson(tid, url, feature_collection, timestamps, index=0):
    
    matchings_dict = match_json(url)
    
    # convert the matchings from the dictionary into GeoJson formatted features
    for matching in matchings_dict['matchings']:
        confidence = matching['confidence']
        # initialize a list to store all the timestamps associated with the matching
        timestamps_list = []  
        feature = {
                   'type': 'Feature',
                   'id': index,
                   'geometry': {
                                'type': 'Point',
                                'coordinates': []
                               },
                   'properties': {
                                'timestamp': None,
                                'confidence': confidence,
                                'tid': tid
                                }
                  }
        
        # loop through the legs and steps of the matching to get a single set of coords for the whole matching
        for leg in matching['legs']:
            
            # each input GPS point defines a leg in the matched route, so popping the timestamp for each leg will
            # give the correct timestamp for the leg
            feature['properties']['timestamp'] = timestamps.pop(0)  
            
            # the first point in the first step of the leg provides the coordinate of the leg point
            feature['geometry']['coordinates'] = leg['steps'][0]['geometry']['coordinates'][0] 
        
        # put the feature representing the matching into the feature collection
        feature_collection['features'].append(feature)
        
        # advance the feature id index 
        index += 1
    
    return feature_collection, index + 1  # return the feature collection and the index of the next feature to add to the collection

In [7]:
# return a pandas dataframe for a given trip id, given an enginec
def get_df(tid, engine):
    # get results as a pandas dataframe
    sql = 'SELECT * FROM trip_points WHERE tid = {} ORDER BY location_timestamp'.format(tid)
    df = pd.read_sql_query(sql, con=engine)
    
    # select only the GPS points where the accuracy is better than 25 meters.  This cleans up the input points to
    # the OSRM map matching algorithm and reduces the number of spurious matches
    df = df.loc[df['accuracy'] < 25]
    
    return df
    

In [8]:
# for a given trip id, add each matching to a python dict formatted as a geojson feature collection
def matched_trip_geojson(tid, engine, feature_collection=None, feature_index=0): 
    
    # get all GPS points for the trip from the Postgres database as a pandas dataframe
    df = get_df(tid, engine)
    
    # number of pieces to break the trip into, since OSRM can only match 100 pts at a time.  
    # dividing by 100 - len(df)/100 b/c there is a 1 point overlap between chunks
    chunks = int(math.ceil(len(df) / (100 - len(df)/100)))
    
    if chunks == 0: # i.e. the dataframe is empty
        print('error: 0 length chunk size')
        return feature_collection, feature_index
    
    print('working on trip', tid)
    print('chunks:', chunks)
    
    # number of records in each chunk, because we'd like to keep them evenly sized and we want 1 point overlap
    chunk_size = (len(df) // chunks) + 1  


    index = 0  # index of the point to start with
    
    for i in range(chunks):  # for each chunk
        
        print('working on chunk:', i+1)
        
        if i == chunks - 1: # i.e. if this is the last chunk
            chunk_df = df[index:]  # get the dataframe from the index to the end
        else:
            chunk_df = df[index:index+chunk_size]
            
            # advance the index to the first record of the next chunk, which is the same as the last point of the last chunk
            index += chunk_size - 1
        
        # initialize a python dict in geojson format for the chunk
        chunk_geojson = {'type': 'LineString',
                         'properties': {}}
        chunk_geojson['coordinates'] = list(zip(chunk_df['longitude'], chunk_df['latitude']))
        chunk_geojson['properties']['timestamps'] = list(chunk_df['location_timestamp'])
        chunk_geojson['properties']['accuracies'] = list(chunk_df['accuracy'])
        #pprint(chunk_geojson)
        
        # get coordinate, timestamp, and search radius strings suitable for creating a URL to pass to the OSRM API
        chunk_coord_string, chunk_timestamp_string, chunk_radius_string = geojson_to_OSRM_strings(chunk_geojson)
        #print(chunk_coord_string, chunk_timestamp_string, chunk_radius_string)
        
        # create a url string to pass to the OSRM API
        chunk_match_url = match(chunk_coord_string, timestamps=chunk_timestamp_string, radiuses=chunk_radius_string)
        #print(chunk_match_url)
        
        # if no feature collection dictionary was passed to the function, initialize an empty one
        if feature_collection == None:
            feature_collection = {
                "type": "FeatureCollection", 
                "features": []
            }
        
        # uncomment below to return line geojson of the matching
        feature_collection, feature_index = matchings_geojson(tid, chunk_match_url, feature_collection, timestamps=chunk_geojson['properties']['timestamps'], index=feature_index)
        
        # uncomment below to return point geojson of the start point of each leg (i.e. the matched location of each GPS input point)
        #chunk_match_geojson, feature_index = leg_points_geojson(tid, chunk_match_url, feature_collection, timestamps=chunk_geojson['properties']['timestamps'], index=feature_index)
        
        #pprint(chunk_match_geojson)
        
    return feature_collection, feature_index # output the feature index so we can pass it back to this function and maintain unique id's


In [9]:
# create a database connection engine
engine = create_engine('postgresql://{}:{}@{}:5432/{}'.format(dbUser, dbPass, dbHost, dbName))

In [10]:
# get a list of all the trip ids for car, bus, and vehicle trips as sqlchemy results query to iterate over
# trip_mode 1 = car, 2 = bus, 3=walk, 4=bike, 5=rail 6=other, 7 = vehicle
auto_trips = engine.execute('SELECT * FROM trip WHERE trip_mode IN (4)')

auto_tripids = [r[0] for r in auto_trips]  # trip id is the first value in the output tuple

Before running the next line, you need to make sure Docker is running.  Paste the following command into a command prompt to run the OSRM route matching program using the car profile:

docker run -t -i -p 5000:5000 -v "C:\Users\orms0027\Documents\HappyRoads\OSRM:/data" osrm/osrm-backend osrm-routed --algorithm mld /data/seven_county_metro.osrm

you may need to restart Docker to clear the containers if you are swapping to a different profile

In [48]:
# use the first trip to intitalize the feature collection
test_feature_collection, feature_index = matched_trip_geojson(auto_tripids[0], engine)

working on trip 1546
chunks: 2
working on chunk: 1
working on chunk: 2


In [49]:
# test to make sure the geojson looks right
pprint(test_feature_collection)

{'features': [{'geometry': {'coordinates': [[-93.255423, 44.962913],
                                            [-93.255424, 44.962727],
                                            [-93.255424, 44.962727],
                                            [-93.255014, 44.962727],
                                            [-93.255014, 44.962727],
                                            [-93.255014, 44.962727],
                                            [-93.255014, 44.962727],
                                            [-93.255371, 44.962727],
                                            [-93.255371, 44.962727],
                                            [-93.255371, 44.962727],
                                            [-93.255371, 44.962727],
                                            [-93.254845, 44.962727],
                                            [-93.254235, 44.962736],
                                            [-93.254235, 44.962736],
                                  

In [11]:
# check the length of auto_tripids to make sure it looks right
len(auto_tripids)

520

In [51]:
# get the rest of the trips
for car_trip in auto_tripids[1:]:
    test_feature_collection, feature_index = matched_trip_geojson(car_trip, engine, feature_collection=test_feature_collection, feature_index=feature_index)

working on trip 2128
chunks: 1
working on chunk: 1
working on trip 2137
chunks: 1
working on chunk: 1
working on trip 2209
chunks: 1
working on chunk: 1
working on trip 2303
chunks: 10
working on chunk: 1
error during trip 2303: Could not match the trace.
working on chunk: 2
error during trip 2303: Could not match the trace.
working on chunk: 3
error during trip 2303: Could not match the trace.
working on chunk: 4
working on chunk: 5
working on chunk: 6
working on chunk: 7
working on chunk: 8
error during trip 2303: Could not match the trace.
working on chunk: 9
working on chunk: 10
working on trip 2307
chunks: 5
working on chunk: 1
working on chunk: 2
working on chunk: 3
error during trip 2307: Could not match the trace.
working on chunk: 4
working on chunk: 5
working on trip 5931
chunks: 1
working on chunk: 1
working on trip 5933
chunks: 1
working on chunk: 1
working on trip 8822
chunks: 1
working on chunk: 1
working on trip 8824
chunks: 11
working on chunk: 1
working on chunk: 2
wor

working on chunk: 8
working on trip 34906
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 34908
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 35394
chunks: 4
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on chunk: 4
working on trip 35396
chunks: 1
working on chunk: 1
working on trip 35398
chunks: 3
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on trip 35402
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 35404
chunks: 9
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on chunk: 4
working on chunk: 5
working on chunk: 6
working on chunk: 7
working on chunk: 8
working on chunk: 9
working on trip 35795
chunks: 4
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on chunk: 4
working on trip 35797
chunks: 1
working on chunk: 1
working on trip 35799
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 35842
chunks: 14
working on chunk: 1
working

working on chunk: 3
working on trip 43377
chunks: 1
working on chunk: 1
working on trip 43379
chunks: 1
working on chunk: 1
working on trip 43405
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 43473
chunks: 10
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on chunk: 4
working on chunk: 5
working on chunk: 6
working on chunk: 7
working on chunk: 8
working on chunk: 9
working on chunk: 10
working on trip 43475
chunks: 6
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on chunk: 4
working on chunk: 5
working on chunk: 6
working on trip 43477
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 43511
chunks: 14
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on chunk: 4
working on chunk: 5
working on chunk: 6
working on chunk: 7
working on chunk: 8
working on chunk: 9
working on chunk: 10
working on chunk: 11
working on chunk: 12
working on chunk: 13
working on chunk: 14
working on trip 43565
chunks

working on trip 47290
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 47292
chunks: 3
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on trip 47369
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 47371
chunks: 4
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on chunk: 4
working on trip 47649
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 47651
chunks: 1
working on chunk: 1
working on trip 47653
chunks: 1
working on chunk: 1
working on trip 47655
chunks: 1
working on chunk: 1
error during trip 47655: Number of coordinates needs to be at least two.
working on trip 47657
chunks: 1
working on chunk: 1
working on trip 47659
chunks: 1
working on chunk: 1
working on trip 47661
chunks: 1
working on chunk: 1
working on trip 47663
chunks: 3
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on trip 47823
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 48050
chunks: 1

working on trip 61166
chunks: 3
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on trip 61168
chunks: 3
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on trip 62213
chunks: 1
working on chunk: 1
working on trip 62215
chunks: 4
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on chunk: 4
working on trip 62445
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 62449
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 63134
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 63136
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 63137
chunks: 1
working on chunk: 1
working on trip 65173
chunks: 4
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on chunk: 4
working on trip 65175
chunks: 4
working on chunk: 1
working on chunk: 2
working on chunk: 3
working on chunk: 4
working on trip 65185
chunks: 1
working on chunk: 1
working on trip 65186
chunks: 1
work

working on chunk: 4
working on chunk: 5
working on chunk: 6
working on chunk: 7
working on chunk: 8
working on trip 99550
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 100004
chunks: 1
working on chunk: 1
working on trip 101450
chunks: 1
working on chunk: 1
working on trip 158719
chunks: 1
working on chunk: 1
working on trip 158725
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 159595
chunks: 1
working on chunk: 1
working on trip 159597
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 159598
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 159603
chunks: 1
working on chunk: 1
working on trip 159605
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 159607
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 159611
chunks: 2
working on chunk: 1
working on chunk: 2
working on trip 161513
chunks: 1
working on chunk: 1
working on trip 161521
chunks: 2
working on chunk: 1
working on chunk: 2

In [52]:
# write the ouput geojson to file
with open('test_features.geojson', 'w') as fh:
    json.dump(test_feature_collection, fh)

# open the file as a geopandas dataframe
gdf = gpd.read_file('test_features.geojson', driver='geojson')
gdf.head()

,start_time,end_time,confidence,tid,geometry
0,1476555148940,1476555356000,0.658705,1546,"LINESTRING (-93.25542299999999 44.962913, -93...."
1,1476555441000,1476555466000,0.031351,1546,"LINESTRING (-93.24431300000001 44.96438, -93.2..."
2,1476555466000,1476555616000,0.820377,1546,"LINESTRING (-93.244086 44.965961, -93.24408699..."
3,1477075200974,1477075268573,0.973637,2128,"LINESTRING (-93.257244 44.97053, -93.25726 44...."
4,1477079790118,1477079795118,0.000000,2137,"LINESTRING (-93.262152 44.972605, -93.262152 4..."


In [53]:
# check the length of the gdf
len(gdf)

6664

In [5]:
# open the trip data in the postgres database as a pandas dataframe
sql = 'SELECT * FROM trip WHERE tid IN {}'.format(tuple(auto_tripids))
trip_df = pd.read_sql_query(sql, con=engine)
trip_df.head()

,tid,uid,start_timestamp,end_timestamp,trip_mode,duration_sec,distance_m,happy,tired,stressed,sad,pain,meaningful
0,2648,1999,1477337340003,1477337370011,5,30,58,3,3,2,2,2,1
1,2743,4000,1477326930056,1477328850001,5,1919,17004,5,3,2,2,3,6
2,2745,4000,1477334370023,1477336320005,5,1949,16769,5,2,3,3,2,4
3,7771,4006,1479130470006,1479130770019,5,300,1895,1,6,1,1,1,7
4,8239,4006,1479218520014,1479219450006,5,929,6759,1,1,1,1,1,1


In [9]:
trip_df.to_csv('rail_trips.csv')

In [55]:
# join the geodataframe to the trip data
merged_gdf = gdf.join(trip_df.set_index('tid'), on='tid')
merged_gdf.head()

,start_time,end_time,confidence,tid,geometry,uid,start_timestamp,end_timestamp,trip_mode,duration_sec,distance_m,happy,tired,stressed,sad,pain,meaningful
0,1476555148940,1476555356000,0.658705,1546,"LINESTRING (-93.25542299999999 44.962913, -93....",2278,1476555090009,1476555720014,4,630,2563,0,0,1,3,2,0
1,1476555441000,1476555466000,0.031351,1546,"LINESTRING (-93.24431300000001 44.96438, -93.2...",2278,1476555090009,1476555720014,4,630,2563,0,0,1,3,2,0
2,1476555466000,1476555616000,0.820377,1546,"LINESTRING (-93.244086 44.965961, -93.24408699...",2278,1476555090009,1476555720014,4,630,2563,0,0,1,3,2,0
3,1477075200974,1477075268573,0.973637,2128,"LINESTRING (-93.257244 44.97053, -93.25726 44....",5001,1477075170030,1477075350011,4,179,914,3,2,2,1,1,1
4,1477079790118,1477079795118,0.000000,2137,"LINESTRING (-93.262152 44.972605, -93.262152 4...",5001,1477079790010,1477080150008,4,359,1466,3,1,1,1,1,1


In [56]:
# check the length of the joined file
len(merged_gdf)

6664

In [57]:
# output the new dataframe as an Esri shapefile
merged_gdf.to_file('matched_biking_trips',driver ='ESRI Shapefile')